In [1]:
# Mount Google Drive
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import csv
import numpy as np
import pandas as pd
import sys
from datetime import datetime

In [3]:
# Data Path
data_dir = "gdrive/My Drive/CounterGEDI/Results/"
data_path = os.path.join(data_dir, 'Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1684275355_multi.json')

root_dir = "gdrive/My Drive/Master_Thesis/"

Base_dir = os.path.join(root_dir, 'data/Custom/CONAN_test.csv')
EDOS_dir = os.path.join(root_dir, 'data/Custom/EDOS_sexist.csv')
test_dir = os.path.join(root_dir, 'data/Custom/T8-S10.csv')

save_dir = os.path.join(root_dir, 'predictions/')

In [4]:
def get_datetime(format):
    # datetime object containing current date and time
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt = now.strftime(format)
    return dt

In [11]:
# Read text file 
filenames = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
for filename in filenames:
  print(filename)
  # Get method name
  x = filename.split('_')
  Model_name = 'CounterGeDI_' + x[1]
  if x[-4] == "gedi":
    Model_name = Model_name + '-gedi'

  # Open the original testset
  if x[3] == "CONAN":
    test = pd.read_csv(Base_dir) 
    task = "Base" 
  elif x[3] == "EDOS":
    test = pd.read_csv(EDOS_dir) 
    test = test.rename(columns={"text": "Hate_Speech"})
    task = "Sexism"
  elif x[3] == "SMALL": 
    test = pd.read_csv(test_dir)
    task = "Small" 

  # Make a copy of dataframe
  df_ = test.copy()

  # Open file
  with open(data_dir + filename, 'r') as f:
    
    # Read predictions
    df = pd.read_json(os.path.join(data_dir, filename))
    df = df[~df['samples'].isna()]
    df = df.drop(columns=['params'])
    df = pd.DataFrame(list(df['samples']))

    # merge test set with test labels
    df_new = df_.merge(df, left_on='Hate_Speech', right_on='hatespeech')
    df_new = df_new.drop(columns=['hatespeech'])
    df_new = df_new.rename(columns={'counterspeech_model': 'Prediction'})

    df_new.to_csv(save_dir + Model_name + '_' + task + '_' + x[-2] + ".csv") 

Generator-CONAN_DialoGPT-medium_on_CONAN_huggingface_08,05,2023--12,14_base.json
Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_19,05,2023--04,32_multi.json
Generator-CONAN_DialoGPT-medium_on_SMALL_gedi_huggingface_1686090576_multi.json
Generator-CONAN_DialoGPT-medium_on_EDOS_gedi_huggingface_1686125903_multi.json


In [ ]:
df_new["Prediction"]

0       [Jews have been part of our society for centur...
1       [It's not true that women are less than human,...
2       [It's not true that women are less than human,...
3       [The vast majority of Muslims are unemployed a...
4       [Muslims use acts of violence to punish people...
                              ...                        
2381    [It's not true that all Muslims are bad people...
2382    [abortion is a human right, and should be resp...
2383    [The vast majority of Muslims are peaceful and...
2384    [Why do you say that? Isn't it just hate speec...
2385    [The real problem in this country is the lack ...
Name: Prediction, Length: 2386, dtype: object